# Tạo Dimension và Fact Tables cho Gold Layer

Notebook này sẽ tạo các bảng Dimension và Fact trong Gold layer với Nessie catalog để phục vụ phân tích dữ liệu tuyển sinh.

## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import os

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder.appName("Create_Gold_Dim_Fact_Tables")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    # ===== Cấu hình MinIO (S3-compatible) =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
print("Spark Session đã được khởi tạo với Nessie catalog cho Gold layer!")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/05 15:46:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/05 15:46:50 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/05 15:46:50 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Spark Session đã được khởi tạo với Nessie catalog cho Gold layer!


## 2. Tạo Database/Namespace trong Nessie cho Gold Layer

In [2]:
# Tạo database gold nếu chưa tồn tại
spark.sql("CREATE DATABASE IF NOT EXISTS nessie.gold_tables")
spark.sql("USE nessie.gold_tables")

print("Database 'gold_tables' đã được tạo và đang sử dụng!")

Database 'gold_tables' đã được tạo và đang sử dụng!


## 3. Tạo Dimension Tables

### 3.1. Dim_Time - Bảng thời gian

In [3]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.dim_time (
    timeKey INT,
    day INT,
    month INT,
    year INT
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'dim_time' đã được tạo!")

25/11/05 15:46:59 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Table 'dim_time' đã được tạo!


### 3.2. Dim_Region - Bảng khu vực thi

In [4]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.dim_region (
    regionKey INT,
    regionId STRING,
    regionName STRING
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'dim_region' đã được tạo!")

Table 'dim_region' đã được tạo!


### 3.3. Dim_School - Bảng trường đại học

In [5]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.dim_school (
    schoolKey INT,
    schoolId STRING,
    schoolName STRING,
    province STRING
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'dim_school' đã được tạo!")

Table 'dim_school' đã được tạo!


### 3.4. Dim_Major - Bảng ngành học

In [6]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.dim_major (
    majorKey INT,
    majorId STRING,
    majorName STRING
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'dim_major' đã được tạo!")

Table 'dim_major' đã được tạo!


### 3.5. Dim_Subject - Bảng môn học

In [7]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.dim_subject (
    subjectKey INT,
    subjectId INT,
    subjectName STRING
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'dim_subject' đã được tạo!")

Table 'dim_subject' đã được tạo!


### 3.6. Dim_Subject_Group - Bảng khối thi (tổ hợp môn)

In [8]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.dim_subject_group (
    subjectGroupKey INT,
    subjectGroupId INT,
    subjectGroupName STRING,
    subjectGroupCode STRING,
    subjectCombination STRING
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'dim_subject_group' đã được tạo!")

Table 'dim_subject_group' đã được tạo!


### 3.7. Dim_Selection_Method - Bảng phương thức xét tuyển

In [9]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.dim_selection_method (
    selectionMethodKey INT,
    selectionMethodId INT,
    selectionMethodName STRING
) USING iceberg
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'dim_selection_method' đã được tạo!")

Table 'dim_selection_method' đã được tạo!


## 4. Tạo Fact Tables

### 4.1. Fact_Benchmark - Bảng sự kiện điểm chuẩn

In [10]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.fact_benchmark (
    benchmarkKey INT,
    subjectGroupKey INT,
    timeKey INT,
    majorKey INT,
    schoolKey INT,
    selectionMethodKey INT,
    score FLOAT,
    yearlyScoreGap FLOAT,
    rankAmongMajors INT,
    rankAmongSchools INT
) USING iceberg
PARTITIONED BY (timeKey)
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'fact_benchmark' đã được tạo!")

Table 'fact_benchmark' đã được tạo!


### 4.2. Fact_Score_Distribution_By_Subject - Bảng phân bố điểm theo môn

In [11]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.fact_score_distribution_by_subject (
    SDBSKey INT,
    regionKey INT,
    subjectKey INT,
    timeKey INT,
    scoreThreshold FLOAT,
    quantity FLOAT
) USING iceberg
PARTITIONED BY (timeKey)
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'fact_score_distribution_by_subject' đã được tạo!")

Table 'fact_score_distribution_by_subject' đã được tạo!


### 4.3. Fact_Score_Distribution_By_Subject_Group - Bảng phân bố điểm theo khối thi

In [12]:
spark.sql("""
CREATE TABLE IF NOT EXISTS nessie.gold_tables.fact_score_distribution_by_subject_group (
    SDBSGKey INT,
    regionKey INT,
    subjectGroupKey INT,
    timeKey INT,
    scoreRange FLOAT,
    quantity FLOAT
) USING iceberg
PARTITIONED BY (timeKey)
TBLPROPERTIES (
    'write.format.default' = 'parquet',
    'write.metadata.compression-codec' = 'gzip'
)
""")

print("Table 'fact_score_distribution_by_subject_group' đã được tạo!")

Table 'fact_score_distribution_by_subject_group' đã được tạo!


## 5. Kiểm Tra Các Tables Đã Tạo

In [13]:
# Liệt kê tất cả các tables trong database gold_tables
print("Danh sách các Dimension và Fact tables trong Gold layer:")
tables = spark.sql("SHOW TABLES IN nessie.gold_tables")
tables.show(truncate=False)

Danh sách các Dimension và Fact tables trong Gold layer:
+-----------+----------------------------------------+-----------+
|namespace  |tableName                               |isTemporary|
+-----------+----------------------------------------+-----------+
|gold_tables|dim_major                               |false      |
|gold_tables|dim_region                              |false      |
|gold_tables|dim_school                              |false      |
|gold_tables|dim_selection_method                    |false      |
|gold_tables|dim_subject                             |false      |
|gold_tables|dim_subject_group                       |false      |
|gold_tables|dim_time                                |false      |
|gold_tables|fact_benchmark                          |false      |
|gold_tables|fact_score_distribution_by_subject      |false      |
|gold_tables|fact_score_distribution_by_subject_group|false      |
+-----------+----------------------------------------+-----------+



## 6. Kiểm Tra Schema của Các Tables

In [14]:
# Kiểm tra schema của một số tables quan trọng
print("=== Schema của Fact_Benchmark ===")
spark.sql("DESCRIBE nessie.gold_tables.fact_benchmark").show(truncate=False)

print("\n=== Schema của Dim_School ===")
spark.sql("DESCRIBE nessie.gold_tables.dim_school").show(truncate=False)

print("\n=== Schema của Dim_Major ===")
spark.sql("DESCRIBE nessie.gold_tables.dim_major").show(truncate=False)

=== Schema của Fact_Benchmark ===
+-----------------------+---------+-------+
|col_name               |data_type|comment|
+-----------------------+---------+-------+
|benchmarkKey           |int      |NULL   |
|subjectGroupKey        |int      |NULL   |
|timeKey                |int      |NULL   |
|majorKey               |int      |NULL   |
|schoolKey              |int      |NULL   |
|selectionMethodKey     |int      |NULL   |
|score                  |float    |NULL   |
|yearlyScoreGap         |float    |NULL   |
|rankAmongMajors        |int      |NULL   |
|rankAmongSchools       |int      |NULL   |
|# Partition Information|         |       |
|# col_name             |data_type|comment|
|timeKey                |int      |NULL   |
+-----------------------+---------+-------+


=== Schema của Dim_School ===
+----------+---------+-------+
|col_name  |data_type|comment|
+----------+---------+-------+
|schoolKey |int      |NULL   |
|schoolId  |string   |NULL   |
|schoolName|string   |NULL   |
